In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from nl2bash.eval import eval_tools, error_analysis
from nl2bash.encoder_decoder.seq2seq.seq2seq_model import Seq2SeqModel
from nl2bash.encoder_decoder import slot_filling
from nl2bash.encoder_decoder import parse_args
from nl2bash.encoder_decoder import meta_experiments
from nl2bash.encoder_decoder import graph_utils
from nl2bash.encoder_decoder import decode_tools
from nl2bash.encoder_decoder import data_utils
import tensorflow as tf
from tqdm import tqdm
import time
import pickle
import numpy as np
import math


import os
import sys

Bashlint grammar set up (124 utilities)



/home/greenmon/.pyenv/versions/3.7.2/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [21]:
import collections
import functools
import os
import pickle
import sys

import numpy as np
import scipy.sparse as ssp
import tensorflow as tf

if sys.version_info > (3, 0):
    from six.moves import xrange

from nl2bash.bashlint import bash, nast, data_tools
from nl2bash.nlp_tools import constants, tokenizer

In [2]:

FLAGS = tf.compat.v1.flags.FLAGS
parse_args.define_input_flags()

In [10]:
FLAGS.data_dir = os.path.join(
    os.path.dirname('/home/greenmon/Projects/nlc2cmd/clai/submission-code/src/submission_code/nlc2cmd/nl2bash/'), "data", FLAGS.dataset)

print("Reading data from {}".format(FLAGS.data_dir))

Reading data from /home/greenmon/Projects/nlc2cmd/clai/submission-code/src/submission_code/nlc2cmd/nl2bash/data/bash


In [11]:
# set up encoder/decider dropout rate
if FLAGS.universal_keep >= 0 and FLAGS.universal_keep < 1:
    FLAGS.sc_input_keep = FLAGS.universal_keep
    FLAGS.sc_output_keep = FLAGS.universal_keep
    FLAGS.tg_input_keep = FLAGS.universal_keep
    FLAGS.tg_output_keep = FLAGS.universal_keep
    FLAGS.attention_input_keep = FLAGS.universal_keep
    FLAGS.attention_output_keep = FLAGS.universal_keep

## Loading Data

In [12]:
train_set, dev_set, test_set = data_utils.load_data(
    FLAGS, use_buckets=False, load_features=True)

Loading data from /home/greenmon/Projects/nlc2cmd/clai/submission-code/src/submission_code/nlc2cmd/nl2bash/data/bash
source vocabulary size = 1489
target vocabulary size = 1109
max source token size = 27
max target token size = 55
source file: /home/greenmon/Projects/nlc2cmd/clai/submission-code/src/submission_code/nlc2cmd/nl2bash/data/bash/train.nl.filtered
target file: /home/greenmon/Projects/nlc2cmd/clai/submission-code/src/submission_code/nlc2cmd/nl2bash/data/bash/train.cm.filtered
source tokenized sequence file: /home/greenmon/Projects/nlc2cmd/clai/submission-code/src/submission_code/nlc2cmd/nl2bash/data/bash/train.nl.token
target tokenized sequence file: /home/greenmon/Projects/nlc2cmd/clai/submission-code/src/submission_code/nlc2cmd/nl2bash/data/bash/train.cm.token
9985 data points read.
max_source_length = 44
max_target_length = 64
source vocabulary size = 1489
target vocabulary size = 1109
max source token size = 27
max target token size = 55
source file: /home/greenmon/Projec

## Preparing Data

In [14]:
def read_parallel_data(nl_path, cm_path):
    with open(nl_path, encoding='utf-8') as f:
        nl_list = [nl.strip() for nl in f.readlines()]
    with open(cm_path, encoding='utf-8') as f:
        cm_list = [cm.strip() for cm in f.readlines()]
    return nl_list, cm_list

In [17]:
split = 'dev'
data_dir = FLAGS.data_dir

print("Split - {}".format(split))
nl_path = os.path.join(data_dir, split + '.nl.filtered')
cm_path = os.path.join(data_dir, split + '.cm.filtered')
nl_list, cm_list = read_parallel_data(nl_path, cm_path)

Split - dev


In [20]:
nl_path = os.path.join(data_dir, split + '.nl.filtered')
cm_path = os.path.join(data_dir, split + '.cm.filtered')
nl_list, cm_list = read_parallel_data(nl_path, cm_path)
print(nl_list[:3])
print(cm_list[:3])

['Adds execution permissions on a script ./etc/bash_completion within Homebrew home folder path.', 'Add prefix like number and "^M${LOGFILE}> " to every non-blank line received on standard input', 'all .jpg or .png images modified in the past week']
['chmod +x $(brew --prefix)/etc/bash_completion', 'nl -s"^M${LOGFILE}>  "', "find . -mtime -7 \\( '*.jpg' -o -name '*.png' \\)"]


In [26]:
def parallel_data_to_tokens(nl_list, cm_list):
    nl_data = [nl_to_tokens(nl, tokenizer.basic_tokenizer) for nl in nl_list]
    cm_data = [cm_to_tokens(cm, data_tools.bash_tokenizer) for cm in cm_list]
    return nl_data, cm_data

def nl_to_tokens(s, tokenizer, to_lower_case=True, lemmatization=True):
    """
    Split a natural language string into a sequence of tokens.
    """
    tokens, _ = tokenizer(
        s, to_lower_case=to_lower_case, lemmatization=lemmatization)
    return tokens

def cm_to_tokens(s, tokenizer, loose_constraints=True, arg_type_only=False,
                 with_prefix=False, with_flag_argtype=True):
    """
    Split a command string into a sequence of tokens.
    """
    tokens = tokenizer(s, loose_constraints=loose_constraints,
                       arg_type_only=arg_type_only,
                       with_prefix=with_prefix,
                       with_flag_argtype=with_flag_argtype)
    return tokens


In [27]:
def prepare_channel(data_dir, nl_list, cm_list, split, channel,
                    parallel_data_to_tokens):
    print("    channel - {}".format(channel))
    # Tokenize data
    nl_tokens, cm_tokens = \
        parallel_data_to_tokens(nl_list, cm_list)
    
    print(cm_tokens)
    
    save_channel_features_to_file(data_dir, split, channel, nl_tokens, cm_tokens,
                                  feature_separator=TOKEN_SEPARATOR)
    # Create or load vocabulary
    nl_vocab_path = os.path.join(data_dir, 'nl.vocab.{}'.format(channel))
    cm_vocab_path = os.path.join(data_dir, 'cm.vocab.{}'.format(channel))
    nl_vocab = create_vocabulary(nl_vocab_path, nl_tokens) \
        if split == 'train' else initialize_vocabulary(nl_vocab_path)[0]
    cm_vocab = create_vocabulary(cm_vocab_path, cm_tokens) \
        if split == 'train' else initialize_vocabulary(cm_vocab_path)
    nl_ids = [tokens_to_ids(data_point, nl_vocab) for data_point in nl_tokens]
    cm_ids = [tokens_to_ids(data_point, cm_vocab) for data_point in cm_tokens]
    save_channel_features_to_file(data_dir, split, 'ids.{}'.format(channel),
                                  nl_ids, cm_ids, feature_separator=' ')

    
    alignments = compute_alignments(
        data_dir, nl_tokens, cm_tokens, split, channel)
    with open(os.path.join(data_dir, '{}.{}.align'.format(split, channel)),
              'wb') as o_f:
        pickle.dump(alignments, o_f)

### To change:
    - result of bash tokenizer (to annotate types)
        - ast2token (dataset construction) vs. ast2command (when decoding)
    - vocabulary


In [28]:
prepare_channel(data_dir, nl_list, cm_list, split, channel='token',
                        parallel_data_to_tokens=parallel_data_to_tokens)

    channel - token
[[], ['nl', '-s<FLAG_SUFFIX>Regex', '"^M${LOGFILE}>  "'], ['find', '.', '-mtime<FLAG_SUFFIX>Timespan', '-7', '\\(', "'*.jpg'", '-or', '-name<FLAG_SUFFIX>Regex', "'*.png'", '\\)'], ['yes', '|', 'cp', '*', '/tmp'], [], [], ['rsync', '-a<FLAG_SUFFIX>', '-v<FLAG_SUFFIX>', '-z<FLAG_SUFFIX>', '-r<FLAG_SUFFIX>', '-u<FLAG_SUFFIX>', '--delete-excluded<FLAG_SUFFIX>', '/media/Incoming/music/', 'server:/media/10001/music/'], ['rsync', '-a<FLAG_SUFFIX>', '-v<FLAG_SUFFIX>', '-z<FLAG_SUFFIX>', '--ignore-existing<FLAG_SUFFIX>', '/source', 'folder/*', 'user@remoteserver:/dstfolder/'], [], [], ['rsync', '-a<FLAG_SUFFIX>', '-u<FLAG_SUFFIX>', '-v<FLAG_SUFFIX>', '-e<FLAG_SUFFIX>', '"ssh -p 2222"', '.', 'me@localhost:/some/path'], ['rsync', '-a<FLAG_SUFFIX>', '-v<FLAG_SUFFIX>', '.', 'server2::sharename/B'], [], ['find', '~/tmp', '-mtime<FLAG_SUFFIX>Timespan', '0', '-exec<FLAG_SUFFIX>UTILITY', 'du', '-k<FLAG_SUFFIX>', '-s<FLAG_SUFFIX>', '{}', '\\;', '|', 'cut', '-f<FLAG_SUFFIX>Number', '1

NameError: name 'save_channel_features_to_file' is not defined